**Installing Necessary Library**

In [5]:
pip install panda


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import json
import pandas as pd
from pathlib import Path
import os
from datetime import datetime

In [7]:
path = str(Path().absolute()).replace("\\","/")
require_path = path + "/Template (DO NOT TOUCH)/"
dir_list= "Template (DO NOT TOUCH)/"+ os.listdir(require_path)[0]
dir_list

'Template (DO NOT TOUCH)/Glenwood Queue - v1 (Template).json'

In [8]:
# simple hour convertion function format from 12hour to 24hour time
def time_convert(hour):
    hour = "0"+hour
    t = datetime.strptime(hour, "%I %p").strftime("%H:%M:%S")
    return(t)
def time_zone(zone):
    if zone == "central" or zone =="Central":
        r = "America/Chicago"
    elif zone == "pacific" or zone == "Pacific":
        r = "America/Los_Angeles"
    elif zone == "east" or zone == "East":
        r = "America/New_York"
    
    return (r)

#import template file
with open(dir_list) as f:
    data = f.read()

queue_name = input("Please Enter The New Queue name only:")
from_hour = input("Please Enter Starting Hours (1-12): ") + " AM"
end_hour = input(("Please Enter Ending Hours (1-12): ")) + " PM"
region = input("Please Enter Time Zone (Central/Pacific/East): ")



from_hour = time_convert(from_hour)
end_hour = time_convert(end_hour)
region = time_zone(region)

print("Queue Name: ", queue_name)
print("Business Hours: ",from_hour + "AM - " + end_hour,"PM")
print("Queue Region Time: ", region)

Head = data[:254].replace("city of santa clara", queue_name)
Tail= data[254:].replace("city of santa clara", queue_name.lower())
data = Head + Tail
data = data.replace("07:00:00", from_hour)
data = data.replace("19:00:00", end_hour)
data = data.replace("America/Chicago",region)

#Export New File
with open("Completed/"+queue_name+" - v1.json", 'w') as outfile:
    outfile.write(data)

Queue Name:  Hillsboro
Business Hours:  07:00:00AM - 19:00:00 PM
Queue Region Time:  America/Chicago


**Check to see if the ring group is replace correctly**

In [9]:
with open("Completed/"+queue_name+" - v1.json") as f:
    data = f.read()
    data = json.loads(data)
    
df=pd.json_normalize(data["steps"])

df["properties.assignment_parameters.ring_groups.ring_groups_list"].value_counts()


properties.assignment_parameters.ring_groups.ring_groups_list
[glenwood management spanish]    11
[glenwood management]            11
Name: count, dtype: int64

**Check for Time/Days/Timezone**

In [29]:
pd.set_option('max_colwidth', None)

for i,value in df[df["component.name"]=="time_based_rules-NDU4NTVlNz"]["exits"].items():
    df_time=pd.json_normalize(value[1])
    for i,value in df_time["condition.ranges"].items():
        time=pd.json_normalize(value)
    for i,value in df_time["condition.timezone"].items():
        time["Time Zone"] = value.split("/")[1].replace("_"," ")

time


,days,time.to,time.from,Time Zone
0,"[MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY, SUNDAY]",17:00:00,08:00:00,New York


In [ ]:
df["component.name"].value_counts()

component.name
play_audio-NjFkZDU2MG              22
voicemail-NGQ0ZDE5Nj               10
callback-OTU1MjIyYm                10
standard_ivr-M2NlYzMzNz             9
calendar_based_rules-OTQ1M2UxMG     4
assignment_and_dial-M2JhZTViYT      2
time_based_rules-NDU4NTVlNz         2
inbound_voice-ZjE1ZjM0MG            1
record-ZGE1ZDUyOD                   1
Name: count, dtype: int64